# Proyecto Sprint 11

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
df_data_0 = pd.read_csv('geo_data_0.csv')
df_data_1 = pd.read_csv('geo_data_1.csv')
df_data_2 = pd.read_csv('geo_data_2.csv')

In [3]:
print(df_data_0.info())
print()
print(df_data_0.shape)
print()
print(df_data_0.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

(100000, 5)

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647


# Entrenamiento y prueba del modelo

In [4]:
def model(data):
    features = data.drop(['product', 'id'], axis=1)
    target = data['product']

    X_train, X_valid, y_train, y_valid = train_test_split(
        features, target, test_size=0.25, random_state=42
    )

    model = LinearRegression()
    model.fit(X_train, y_train)

    predictions = model.predict(X_valid)

    rmse = mean_squared_error(y_valid, predictions, squared=False)
    mean_pred = predictions.mean()

    return {
        'model': model,
        'predictions': predictions,
        'real': y_valid.reset_index(drop=True),
        'rmse': rmse,
        'mean_prediction': mean_pred
    }

Esta función:

    Divide los datos en entrenamiento y validación

    Entrena el modelo de regresión lineal

    Calcula predicciones

    Devuelve las predicciones, las verdaderas respuestas, el RMSE y el valor promedio predicho

In [5]:
resultados_0 = model(df_data_0)
resultados_1 = model(df_data_1)
resultados_2 = model(df_data_2)

In [6]:
def mostrar_resultados(nombre_region, resultados):
    print(f"--- {nombre_region} ---")
    print(f"RMSE: {resultados['rmse']:.2f}")
    print(f"Volumen medio de reservas predicho: {resultados['mean_prediction']:.2f} mil barriles")
    print()

mostrar_resultados("Región 0", resultados_0)
mostrar_resultados("Región 1", resultados_1)
mostrar_resultados("Región 2", resultados_2)

--- Región 0 ---
RMSE: 37.76
Volumen medio de reservas predicho: 92.40 mil barriles

--- Región 1 ---
RMSE: 0.89
Volumen medio de reservas predicho: 68.71 mil barriles

--- Región 2 ---
RMSE: 40.15
Volumen medio de reservas predicho: 94.77 mil barriles



Región 1 tiene el modelo más preciso (RMSE), lo cual sugiere que la relación entre las características y el volumen de reservas es más fuerte o más lineal ahí. Sin embargo, su volumen medio es bajo: 68.71 mil barriles por pozo, lo que es muy por debajo del umbral de rentabilidad (111.1 mil barriles por pozo).

Región 2 tiene el mayor volumen medio (94.77), aunque su modelo es menos preciso.

Región 0 también tiene un buen volumen medio (92.40), pero el modelo es más inexacto que en la Región 1 y solo levemente mejor que en la 2.

# Cálculo de ganancias

Seleccionar los 200 pozos con las mayores predicciones de reservas (según nuestro modelo) y calcular el beneficio esperado de explotarlos.
Cada unidad del volumen predicho representa mil barriles, y cada barril genera 4.5 USD, por lo que:

    Beneficio por unidad de producto = 4.5 × 1000 = 4500 USD

Entonces, el beneficio total se calcula así:

    Beneficio = suma de las reservas reales de los 200 mejores pozos × 4500 USD
   Presupuesto total: 100 millones USD
Como abrimos 200 pozos, cada pozo no debe costar más de 500 mil USD para que el proyecto sea rentable.

Para cada región:

    Usamos las predicciones del conjunto de validación.

    Ordenamos las predicciones de mayor a menor.

    Seleccionamos las 200 mayores predicciones.

    Obtenemos las reservas reales correspondientes a esos pozos.

    Calculamos el beneficio total:
    Reservas reales seleccionadas×4500Reservas reales seleccionadas×4500

Esto nos dirá si, al elegir estratégicamente los mejores pozos, una región puede superar el umbral de rentabilidad.

In [7]:
def top_200(predicciones, reales):
    import numpy as np
    df_temp = pd.DataFrame({
        'pred': predicciones,
        'real': reales
    })

    top_200 = df_temp.sort_values(by='pred', ascending=False).head(200)

    beneficio = top_200['real'].sum() * 4500
    
    volumen_medio = top_200['real'].mean()

    return beneficio, volumen_medio

In [8]:
beneficio_0, volumen_medio_0 = top_200(resultados_0['predictions'], resultados_0['real'])
beneficio_1, volumen_medio_1 = top_200(resultados_1['predictions'], resultados_1['real'])
beneficio_2, volumen_medio_2 = top_200(resultados_2['predictions'], resultados_2['real'])

In [9]:
print("--- Beneficios potenciales por región ---")
print(f"Región 0: Beneficio estimado = ${beneficio_0:,.0f}, Volumen medio = {volumen_medio_0:.2f} mil barriles")
print(f"Región 1: Beneficio estimado = ${beneficio_1:,.0f}, Volumen medio = {volumen_medio_1:.2f} mil barriles")
print(f"Región 2: Beneficio estimado = ${beneficio_2:,.0f}, Volumen medio = {volumen_medio_2:.2f} mil barriles")

--- Beneficios potenciales por región ---
Región 0: Beneficio estimado = $133,591,411, Volumen medio = 148.43 mil barriles
Región 1: Beneficio estimado = $124,150,867, Volumen medio = 137.95 mil barriles
Región 2: Beneficio estimado = $125,985,718, Volumen medio = 139.98 mil barriles


Todas las regiones superan el umbral de rentabilidad (111.1 mil barriles por pozo), lo cual es excelente. Significa que el proyecto es viable si se eligen estratégicamente los pozos.

Región 0 es la más rentable, tanto en beneficio estimado como en volumen medio producido por pozo.

Aunque Región 1 tiene el modelo más preciso (menor RMSE), su volumen de producción es menor que en las otras dos, por lo que su rentabilidad no es la mejor.

Región 2 es un buen segundo lugar, con mejor volumen que la 1, pero menos beneficio que la 0.

# Riesgos y ganancias para cada región

In [10]:
def bootstrap_riesgo(serie_real, n_iter=1000, n_pozo=200, precio_por_unidad=4500):
    import numpy as np

    ganancias = []

    for i in range(n_iter):
        muestra = serie_real.sample(n_pozo, replace=True)
        ganancia = muestra.sum() * precio_por_unidad
        ganancias.append(ganancia)

    ganancias = np.array(ganancias)

    promedio = np.mean(ganancias)
    intervalo = np.percentile(ganancias, [2.5, 97.5])
    riesgo_perdida = (ganancias < 100_000_000).mean() * 100

    return promedio, intervalo, riesgo_perdida

In [11]:
riesgo_0 = bootstrap_riesgo(resultados_0['real'].sort_values(ascending=False).head(200))
riesgo_1 = bootstrap_riesgo(resultados_1['real'].sort_values(ascending=False).head(200))
riesgo_2 = bootstrap_riesgo(resultados_2['real'].sort_values(ascending=False).head(200))

In [12]:
def mostrar_riesgos(nombre, resultados):
    promedio, intervalo, riesgo = resultados
    print(f"--- {nombre} ---")
    print(f"Beneficio promedio: ${promedio:,.0f}")
    print(f"Intervalo de confianza 95%: ${intervalo[0]:,.0f} - ${intervalo[1]:,.0f}")
    print(f"Riesgo de pérdida: {riesgo:.2f}%")
    print()

mostrar_riesgos("Región 0", riesgo_0)
mostrar_riesgos("Región 1", riesgo_1)
mostrar_riesgos("Región 2", riesgo_2)

--- Región 0 ---
Beneficio promedio: $165,092,478
Intervalo de confianza 95%: $164,955,960 - $165,242,411
Riesgo de pérdida: 0.00%

--- Región 1 ---
Beneficio promedio: $124,150,867
Intervalo de confianza 95%: $124,150,867 - $124,150,867
Riesgo de pérdida: 0.00%

--- Región 2 ---
Beneficio promedio: $169,470,744
Intervalo de confianza 95%: $169,348,917 - $169,596,706
Riesgo de pérdida: 0.00%



# Conclusión:

En el análisis de los mejores 200 pozos por región, se seleccionó la región 0 debido a su beneficio estimado más alto en la selección de los 200 mejores pozos, con un beneficio de $133,591,411. 

Sin embargo, en el análisis de bootstrapping, donde se simulan diferentes escenarios con una mayor variabilidad, la región 2 resultó ser la mejor opción, mostrando un beneficio promedio más alto de $169,471,676.

#### Coincidencia con la elección anterior:
- No coincide: En el análisis de los 200 mejores pozos, se eligió la región 0, mientras que en el análisis de bootstrapping, la región 2 mostró el mayor beneficio promedio y fue seleccionada.

#### Justificación de la diferencia:
- El análisis de los mejores 200 pozos por región se basa en la selección con las predicciones más altas, lo que genera una estimación más estable del beneficio.
- El análisis de bootstrapping incorpora más variabilidad, lo que permite capturar una distribución más realista de los beneficios y refleja una visión más completa del riesgo y beneficio.

#### Conclusión final:
Aunque la elección en el análisis de **bootstrapping** no coincide con la del análisis de los 200 mejores pozos, la **región 2** sigue siendo la opción más atractiva debido a su mayor **beneficio promedio** y su **distribución confiable** de los beneficios. Esto demuestra que el análisis de **bootstrapping** proporciona una evaluación más robusta y realista del riesgo y beneficio, siendo una herramienta valiosa para la toma de decisiones en la selección de la región óptima.

